# Upsample
This notebook loads the reconstructed data and upsamples to 15-minute frequency.

## 0 - Setup

### 0.1 - Imports
Load the necessary dependencies.

In [1]:
from ydata.connectors import LocalConnector, GCSConnector
from ydata.utils.formats import read_json

/opt/conda/lib/python3.7/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


## 0.2 - Auxiliary Functions
The auxiliary functions are custom-designed utilities developed for the REE use case.

In [2]:
from utils import preprocess_data
from upsample import upsample_stations
from imputation import resample

## 1 - Load Data

In [3]:
# Create the Local Connector
connector = LocalConnector()

In [4]:
# Load the reconstructed data for the given device
reconstructed = preprocess_data(connector.read_file('whole_year_reconstructed.csv'))

## 2 - Upsample
For each device, upsample the timeseries frequency from 1H to 15 minutes.

In [5]:
reconstructed_upsampled = upsample_stations(reconstructed)

## 3 - Upsample Original
For the original values, the upsample is done by resampling the original values with 15 minute frequency instead of a random walk upsampling.

In [6]:
# Load the original data for the given device
original = preprocess_data(connector.read_file('train_allmeters.csv'))

In [7]:
# Resample the original data for 15 minute frequency
original_upsampled = resample(original, ts_col='timestamp', ts_freq='15T', partition_by='station')

## 4 - Store Data
Load the reconstruction and the original values with 15 minute frequency.

In [ ]:
# Load the credentials
credentials = read_json('gcs_credentials.json')

# Create the connector for Google Cloud Storage
connector = GCSConnector('ydatasynthetic', gcs_credentials=credentials)

# Store the reconstructed data upsampled to 15 minutes
connector.write_file(reconstructed_upsampled, 'gs://pipelines_artifacts/wind_measurements_pipeline/outputs/reconstructed_upsampled.csv', index=True)

# Store the original data upsampled to 15 minutes
connector.write_file(original_upsampled, 'gs://pipelines_artifacts/wind_measurements_pipeline/outputs/original_upsampled.csv', index=True)